In [1]:
import numpy as np
import tifffile
import napari 
import os
from skimage.measure import regionprops, label

from scripts.config_model import save_experiment_config, tree
from scripts.sample_db import SampleDB
from scripts.utils.image_utils import load_tiff_as_hyperstack, save_array_as_hyperstack_tiff

<Figure size 640x480 with 0 Axes>

In [2]:
# Function to apply transformation
def apply_transformation(points, transform):
    rotation = Rotation.from_matrix(transform[:3, :3])
    translation = transform[:3, 3]
    return rotation.apply(points) + translation


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    x, y, z = np.round(point).astype(int)
    if 0 <= x < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= z < mask.shape[2]:
        return mask[x, y, z]
    return False


In [3]:
# Step 1: Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Step 2: Load experiment configuration
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)
print(exp.sample.id)


20220426_RM0008_130hpf_fP1_f3


In [4]:
# Step 3: Load images and transformation matrices

#LM trials images
processed_folder = os.path.join(exp.paths.trials_path, "processed")

lm_trials_images_path = os.path.join(processed_folder, f"sum_raw_trials_{exp.sample.id}.tif")
lm_trials_images =tifffile.imread(lm_trials_images_path)
lm_trial = lm_trials_images[6,-1]
print(lm_trial.shape)

#LM trials masks
lm_planes_masks_path = os.path.join(exp.paths.trials_path,"masks",f"masks_{exp.sample.id}_params_cp_-1-ft_0.4-st_0.01-resample_True_augment=False.tif")
lm_planes_masks = tifffile.imread(lm_planes_masks_path)
lm_mask = lm_planes_masks[6,-1] 
print(lm_mask.shape)

lm_plane_props = regionprops(label(lm_mask))
lm_plane_centroids =  np.array([prop.centroid for prop in lm_plane_props])

#EM warped stack
em_warped_stack_path = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_em_stack_cropped_woResin_rough_rotated_to_LM_fov_lm_res_em.tif')
em_warped_stack = tifffile.imread(em_warped_stack_path)


(256, 512)
(256, 512)


In [5]:
lm_plane_centroids.shape

(111, 2)

In [6]:
# Transformation matrices
t_lm_trial_to_lm_stack_path = os.path.join(processed_folder, "registration_tform_lm_plane06_lm_stack.npy")
t_lm_trial_to_lm_stack = np.load(t_lm_trial_to_lm_stack_path)

In [7]:
t_lm_trial_to_lm_stack.shape

(4, 4)

In [8]:
transformed_lm_centroids = apply_transformation(lm_plane_centroids, t_lm_trial_to_lm_stack)

NameError: name 'Rotation' is not defined

In [26]:
#Visualize
viewer = napari.Viewer()
viewer.add_image(lm_trial, name="lm_trial")
viewer.add_labels(label(lm_mask), name="lm_mask")

D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Labels layer 'lm_mask' at 0x1c10ebdba90>

In [35]:
viewer.add_image(em_warped_stack, name="em_warped_stack")

<Image layer 'em_warped_stack' at 0x1c12f747ee0>

In [42]:
viewer.add_points(lm_plane_centroids, name="lm_plane_centroids")

<Points layer 'lm_plane_centroids' at 0x1c10eeb7dc0>

In [11]:
os.listdir(os.path.join(exp.paths.trials_path,"masks"))

['masks_20220426_RM0008_130hpf_fP1_f3.tif',
 'masks_20220426_RM0008_130hpf_fP1_f3_cp_-3-ft_0-st_0.01-resample_True-augment_False.tif',
 'masks_20220426_RM0008_130hpf_fP1_f3_params_cp_-1-ft_0.4-st_0.01-resample_True_augment=False.tif']

In [ ]:
#######################################################

In [1]:
import numpy as np
import pandas as pd
import napari
from scipy.spatial.transform import Rotation
import tifffile

# Function to apply transformation
def apply_transformation(points, transform):
    rotation = Rotation.from_matrix(transform[:3, :3])
    translation = transform[:3, 3]
    return rotation.apply(points) + translation


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    x, y, z = np.round(point).astype(int)
    if 0 <= x < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= z < mask.shape[2]:
        return mask[x, y, z]
    return False


# Load transformation matrix for lm trials to lm_stack
t_lm_trials_to_lm_stack = np.load("t_lm_trials_to_lm_stack.npy")

# Load centroids of lm plane
lm_trials_centroids = np.load("lm_trials_centroids.npy")

# Load em warped mask
em_warped_mask = tifffile.imread("em_warped_mask.tif")

# Apply transformations to lm_plane_centroids to fit lm_stak and to ultimately fit to em_stack 

centroids_in_lm_stack = apply_transformation(lm_plane_centroids, lm_to_lm_stack)
centroids_in_em_stack = apply_transformation(centroids_in_lm_stack, lm_stack_to_em_stack)


# Identify  centroids within the 3D mask

centroids_in_mask = [point for point in centroids_in_em_stack if is_within_mask(point, em_mask)]


# Validate by visual inspection in napari (Found cells in green)
# Create Napari viewer
viewer = napari.Viewer()

# Add layers to the viewer
viewer.add_image(em_mask, name='EM Mask')
viewer.add_points(points_layer, size=5, face_color='green', name='Found Cells')

# Run Napari viewer
napari.run()


FileNotFoundError: [Errno 2] No such file or directory: 't_lm_trials_to_lm_stack.npy'